In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import time
import json
import string
from datetime import date 
from scipy.stats import linregress

#Notes: https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7



In [2]:
#Import the API Key
#from api_key import api_key
#Don't need API Key

#import data
#url = "https://data.cdc.gov/resource/hn4x-zwk7.json"
data_df = pd.read_csv(r"Resources\Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System_20231101.csv")

#response = requests.get(url)
#data = response.json()

#data_df.info()
data_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2020,2020,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,NaN,PA,PA1,Q047,VALUE,59,Race/Ethnicity,Hispanic,RACE,RACEHIS
1,2014,2014,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
2,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q036,VALUE,59,Income,"$50,000 - $74,999",INC,INC5075
3,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Income,Data not reported,INC,INCNR
4,2015,2015,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,NaN,Value,...,NaN,PA,PA1,Q045,VALUE,59,Income,"Less than $15,000",INC,INCLESS15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88624,2021,2021,ND,North Dakota,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming vegetab...,NaN,Value,...,"(47.47531977900047, -100.11842104899966)",FV,FV1,Q019,VALUE,38,Gender,Male,GEN,MALE
88625,2021,2021,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q047,VALUE,72,Income,"$35,000 - $49,999",INC,INC3550
88626,2021,2021,WI,Wisconsin,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(44.39319117400049, -89.81637074199966)",PA,PA1,Q047,VALUE,55,Income,"$75,000 or greater",INC,INC75PLUS
88627,2021,2021,UT,Utah,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(39.360700171000474, -111.58713063499971)",FV,FV1,Q018,VALUE,49,Income,"$50,000 - $74,999",INC,INC5075


In [3]:

list_of_categories = data_df['StratificationCategory1'].unique()
list_of_categories


array(['Race/Ethnicity', 'Education', 'Income', 'Age (years)', 'Gender',
       'Total', nan], dtype=object)

In [4]:
ethnicity_df = [data_df['StratificationCategory1'] == 'Race/Ethnicity']
ethnicity_df

[0         True
 1        False
 2        False
 3        False
 4        False
          ...  
 88624    False
 88625    False
 88626    False
 88627    False
 88628    False
 Name: StratificationCategory1, Length: 88629, dtype: bool]

In [5]:
education_df = data_df[data_df['StratificationCategory1'] == 'Education']
education_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
1,2014,2014,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
7,2012,2012,DC,District of Columbia,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(38.890371385, -77.031961127)",OWS,OWS1,Q036,VALUE,11,Education,Less than high school,EDU,EDUHS
10,2015,2015,GU,Guam,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(13.444304, 144.793731)",PA,PA1,Q047,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
21,2012,2012,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Education,Some college or technical school,EDU,EDUCOTEC
43,2016,2016,RI,Rhode Island,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(41.708280193, -71.522470314)",OWS,OWS1,Q036,VALUE,44,Education,College graduate,EDU,EDUCOGRAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88571,2021,2021,DC,District of Columbia,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(38.89037138500049, -77.03196112699965)",FV,FV1,Q018,VALUE,11,Education,Less than high school,EDU,EDUHS
88580,2021,2021,UT,Utah,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(39.360700171000474, -111.58713063499971)",PA,PA1,Q047,VALUE,49,Education,College graduate,EDU,EDUCOGRAD
88581,2021,2021,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q037,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
88615,2021,2021,VT,Vermont,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(43.62538123900049, -72.51764079099962)",OWS,OWS1,Q037,VALUE,50,Education,Some college or technical school,EDU,EDUCOTEC


In [6]:
income_df = data_df[data_df['StratificationCategory1'] == 'Income']
income_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
2,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q036,VALUE,59,Income,"$50,000 - $74,999",INC,INC5075
3,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Income,Data not reported,INC,INCNR
4,2015,2015,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,NaN,Value,...,NaN,PA,PA1,Q045,VALUE,59,Income,"Less than $15,000",INC,INCLESS15
8,2015,2015,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q047,VALUE,72,Income,"$25,000 - $34,999",INC,INC2535
13,2012,2012,WY,Wyoming,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(43.235541343, -108.109830353)",PA,PA1,Q047,VALUE,56,Income,"Less than $15,000",INC,INCLESS15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88620,2021,2021,WY,Wyoming,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming vegetab...,NaN,Value,...,"(43.23554134300048, -108.10983035299967)",FV,FV1,Q019,VALUE,56,Income,"$15,000 - $24,999",INC,INC1525
88625,2021,2021,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q047,VALUE,72,Income,"$35,000 - $49,999",INC,INC3550
88626,2021,2021,WI,Wisconsin,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(44.39319117400049, -89.81637074199966)",PA,PA1,Q047,VALUE,55,Income,"$75,000 or greater",INC,INC75PLUS
88627,2021,2021,UT,Utah,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(39.360700171000474, -111.58713063499971)",FV,FV1,Q018,VALUE,49,Income,"$50,000 - $74,999",INC,INC5075


In [7]:
age_df = data_df[data_df['StratificationCategory1'] == 'Age (years)']
age_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
9,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(32.840571122, -86.631860762)",OWS,OWS1,Q036,VALUE,1,Age (years),25 - 34,AGEYR,AGEYR2534
17,2015,2015,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Age (years),55 - 64,AGEYR,AGEYR5564
24,2015,2015,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(18.220833, -66.590149)",OWS,OWS1,Q037,VALUE,72,Age (years),55 - 64,AGEYR,AGEYR5564
25,2011,2011,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,NaN,PA,PA1,Q047,VALUE,59,Age (years),18 - 24,AGEYR,AGEYR1824
29,2015,2015,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q047,VALUE,72,Age (years),45 - 54,AGEYR,AGEYR4554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88609,2021,2021,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Age (years),45 - 54,AGEYR,AGEYR4554
88610,2021,2021,LA,Louisiana,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(31.31266064400046, -92.44568007099969)",OWS,OWS1,Q037,VALUE,22,Age (years),55 - 64,AGEYR,AGEYR5564
88617,2021,2021,SC,South Carolina,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(33.998821303000454, -81.04537120699968)",FV,FV1,Q018,VALUE,45,Age (years),35 - 44,AGEYR,AGEYR3544
88621,2021,2021,HI,Hawaii,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(21.304850435000446, -157.85774940299973)",FV,FV1,Q018,VALUE,15,Age (years),18 - 24,AGEYR,AGEYR1824


In [8]:
gender_df = data_df[data_df['StratificationCategory1'] == 'Gender']
gender_df


,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
23,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,NaN,PA,PA1,Q047,VALUE,59,Gender,Female,GEN,FEMALE
32,2020,2020,ME,Maine,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(45.254228894000505, -68.98503133599962)",PA,PA1,Q047,VALUE,23,Gender,Male,GEN,MALE
36,2016,2016,WY,Wyoming,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(43.235541343, -108.109830353)",OWS,OWS1,Q037,VALUE,56,Gender,Female,GEN,FEMALE
41,2016,2016,NY,New York,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(42.827001032, -75.543970427)",OWS,OWS1,Q036,VALUE,36,Gender,Male,GEN,MALE
49,2016,2016,AR,Arkansas,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(34.748650124, -92.274490743)",OWS,OWS1,Q037,VALUE,5,Gender,Male,GEN,MALE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88568,2021,2021,CO,Colorado,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(38.843840757000464, -106.13361092099967)",PA,PA1,Q047,VALUE,8,Gender,Female,GEN,FEMALE
88579,2021,2021,MT,Montana,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(47.06652897200047, -109.42442064499971)",OWS,OWS1,Q036,VALUE,30,Gender,Male,GEN,MALE
88597,2021,2021,DE,Delaware,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(39.008830667000495, -75.57774116799965)",OWS,OWS1,Q037,VALUE,10,Gender,Female,GEN,FEMALE
88607,2021,2021,NH,New Hampshire,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(43.65595011300047, -71.50036091999965)",PA,PA1,Q047,VALUE,33,Gender,Female,GEN,FEMALE


In [ ]:
new_data_df = data_df['StratificationCategory1'] == 'Age'

In [18]:
data_df.plot.bar(x='Data_Value', 
                 y=[['StratificationCategory1'] == 'Age_(years)', 
                 ['StratificationCategory1'] == 'Gender', 
                 ['StratificationCategory1'] == 'Race'], figsize=(9,8))
plot.show()             

ValueError: Item wrong length 3 instead of 88629.

In [9]:
totals_df = data_df[data_df['StratificationCategory1'] == 'Total']
totals_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
31,2015,2015,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(18.220833, -66.590149)",OWS,OWS1,Q036,VALUE,72,Total,Total,OVR,OVERALL
53,2016,2016,NM,New Mexico,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(34.520880952, -106.240580985)",OWS,OWS1,Q036,VALUE,35,Total,Total,OVR,OVERALL
76,2015,2015,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in muscle-strengt...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q046,VALUE,72,Total,Total,OVR,OVERALL
93,2016,2016,NH,New Hampshire,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(43.655950113, -71.50036092)",OWS,OWS1,Q036,VALUE,33,Total,Total,OVR,OVERALL
110,2016,2016,AZ,Arizona,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(34.86597028, -111.763811277)",PA,PA1,Q047,VALUE,4,Total,Total,OVR,OVERALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88436,2021,2021,ME,Maine,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(45.254228894000505, -68.98503133599962)",OWS,OWS1,Q036,VALUE,23,Total,Total,OVR,OVERALL
88464,2021,2021,TX,Texas,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(31.827240407000488, -99.42677020599967)",OWS,OWS1,Q037,VALUE,48,Total,Total,OVR,OVERALL
88489,2021,2021,MS,Mississippi,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(32.745510099000455, -89.53803082499968)",FV,FV1,Q018,VALUE,28,Total,Total,OVR,OVERALL
88547,2021,2021,TX,Texas,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(31.827240407000488, -99.42677020599967)",PA,PA1,Q047,VALUE,48,Total,Total,OVR,OVERALL


In [11]:
#response.content



In [11]:
# Convert data into a Pandas DataFrame
#data_df = pd.DataFrame(data)

#data_df = data_df[['yearend','locationabbr','question','data_value','sample_size','age_years','StratificationCategory1','Stratification1']]
data_df = data_df[['YearEnd','LocationAbbr','Question','Data_Value','Sample_Size','Race/Ethnicity','Education','Gender','StratificationCategory1','Stratification1']]
#data_df['data_value'] = data_df.data_value.astype(float)
#data_df['sample_size'] = data_df.sample_size.astype(float)

#data_df = data_df[data_df['sample_size'].notna()]

data_df.info()
# Show Record Count

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88629 entries, 0 to 88628
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YearEnd                  88629 non-null  int64  
 1   LocationAbbr             88629 non-null  object 
 2   Question                 88629 non-null  object 
 3   Data_Value               79851 non-null  float64
 4   Sample_Size              79851 non-null  float64
 5   Race/Ethnicity           25320 non-null  object 
 6   Education                12660 non-null  object 
 7   Gender                   6330 non-null   object 
 8   StratificationCategory1  88620 non-null  object 
 9   Stratification1          88620 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 6.8+ MB


In [12]:
 data_df
    

,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
0,2020,US,Percent of adults who engage in no leisure-tim...,30.6,31255.0,Hispanic,NaN,NaN,Race/Ethnicity,Hispanic
1,2014,GU,Percent of adults aged 18 years and older who ...,29.3,842.0,NaN,High school graduate,NaN,Education,High school graduate
2,2013,US,Percent of adults aged 18 years and older who ...,28.8,62562.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"
3,2013,US,Percent of adults aged 18 years and older who ...,32.7,60069.0,NaN,NaN,NaN,Income,Data not reported
4,2015,US,Percent of adults who achieve at least 300 min...,26.6,30904.0,NaN,NaN,NaN,Income,"Less than $15,000"
...,...,...,...,...,...,...,...,...,...,...
88624,2021,ND,Percent of adults who report consuming vegetab...,27.2,2690.0,NaN,NaN,Male,Gender,Male
88625,2021,PR,Percent of adults who engage in no leisure-tim...,37.4,347.0,NaN,NaN,NaN,Income,"$35,000 - $49,999"
88626,2021,WI,Percent of adults who engage in no leisure-tim...,11.8,850.0,NaN,NaN,NaN,Income,"$75,000 or greater"
88627,2021,UT,Percent of adults who report consuming fruit l...,37.1,1451.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"


In [13]:
#Add calculated number of obese people
data_df['calculated_num_obese'] = round(data_df['Sample_size'] * data_df['data_value'] * .01)
data_df.head()

KeyError: 'Sample_size'

In [16]:
num_of_participants = data_df['sample_size'].sum()
num_of_participants

KeyError: 'sample_size'

In [17]:
#These are the questions this dataset answers
list_of_questions = data_df['question'].unique()
print(sorted(list_of_questions))

KeyError: 'question'

In [11]:
#These are the age groups
age_groups = data_df['age_years'].unique()
print(age_groups)

['18 - 24' '25 - 34' '35 - 44' '45 - 54' '55 - 64' '65 or older' nan]


In [12]:
overweight_df = data_df[data_df['question'] == 'Percent of adults aged 18 years and older who have an overweight classification']
overweight_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
0,2021,AL,Percent of adults aged 18 years and older who ...,28.5,232.0,18 - 24,66.0
7,2021,AL,Percent of adults aged 18 years and older who ...,21.1,433.0,25 - 34,91.0
12,2021,AL,Percent of adults aged 18 years and older who ...,31.8,521.0,35 - 44,166.0
18,2021,AL,Percent of adults aged 18 years and older who ...,33.8,594.0,45 - 54,201.0
22,2021,AL,Percent of adults aged 18 years and older who ...,32.7,826.0,55 - 64,270.0


In [13]:
obese_df = data_df[data_df['question'] == 'Percent of adults aged 18 years and older who have obesity']
obese_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
4,2021,AL,Percent of adults aged 18 years and older who ...,25.4,232.0,18 - 24,59.0
9,2021,AL,Percent of adults aged 18 years and older who ...,46.6,433.0,25 - 34,202.0
13,2021,AL,Percent of adults aged 18 years and older who ...,45.8,521.0,35 - 44,239.0
16,2021,AL,Percent of adults aged 18 years and older who ...,46.4,594.0,45 - 54,276.0
20,2021,AL,Percent of adults aged 18 years and older who ...,44.4,826.0,55 - 64,367.0


In [29]:
no_leisure_df = data_df[data_df['question'] == 'Percent of adults who engage in no leisure-time physical activity']
no_leisure_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 1 to 997
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [28]:
no_fruit_df = data_df[data_df['question'] == 'Percent of adults who report consuming fruit less than one time daily']
no_fruit_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 2 to 998
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [27]:
no_vegetables_df = data_df[data_df['question'] == 'Percent of adults who report consuming vegetables less than one time daily']
no_vegetables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 3 to 995
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [17]:
alabama_df = data_df[data_df['locationabbr'] == 'AL']
alabama_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
0,2021,AL,Percent of adults aged 18 years and older who ...,28.5,232.0,18 - 24,66.0
1,2021,AL,Percent of adults who engage in no leisure-tim...,19.3,254.0,18 - 24,49.0
2,2021,AL,Percent of adults who report consuming fruit l...,44.8,223.0,18 - 24,100.0
3,2021,AL,Percent of adults who report consuming vegetab...,24.7,219.0,18 - 24,54.0
4,2021,AL,Percent of adults aged 18 years and older who ...,25.4,232.0,18 - 24,59.0
